In [1]:
import cobra
import pandas as pd
import json
from copy import deepcopy
from custom_functions_scRBA import *

import sys
sys.path.append('/home/hvdinh16/Workspace/workpy3/common/')
from custom_functions import make_escher_csv

#### FBA optimization python function

In [2]:
def evalModelAtMu(model, coeffs, mu, biomId='BIODIL-BIODILRESPIRATION'):
#def evalModelAtMu(model, coeffs, mu, biomId='BIODIL-BIODILFERMENTATION'):
    # Need to load cobraModel "model", unevaluated symbolic coefficients "coeffs",
    # and growth rate at which the model is evaluated "mu"
    # "Unevaluated symbolic coefficients" means the symbolic expression in string
    # form before being evaluated into number in float form via eval("string")
    
    # Evaluated symbolic coefficients
    coeffs_eval = dict()
    for k,v in coeffs.items():
        coeffs_eval[k] = eval(str(v))
    #coeffs_eval = {k:eval(str(v)) for k,v in coeffs.items()}
    
    # Implemented evaluated symbolic coefficients to model
    for k,v in coeffs_eval.items():
        r,m = k
        rxn = model.reactions.get_by_id(r)
        met = model.metabolites.get_by_id(m)
        rxn.add_metabolites({met:v})
        
    # Set growth rate mu
    model.reactions.get_by_id(biomId).bounds = (mu,mu)
    
    # Run FBA
    fba = model.optimize()
    
    # Remove symbolic coefficients implementation
    for k,v in coeffs_eval.items():
        r,m = k
        rxn = model.reactions.get_by_id(r)
        met = model.metabolites.get_by_id(m)
        rxn.subtract_metabolites({met:v})
    
    return(fba)

In [3]:
def selectCoeffSet(mu, breakpoint, coeffs_low, coeffs_high):
    if mu < breakpoint:
        coeffs = deepcopy(coeffs_low)
    else:
        coeffs = deepcopy(coeffs_high)
    return(coeffs)

#### Prepare model

In [4]:
model = cobra.io.load_json_model('./models/cobraModel_constantStoichOnly_noObj.json')
model.solver = 'cplex'
model.solver.configuration.tolerances.feasibility = 1e-9
model.solver.configuration.tolerances.optimality = 1e-7

#rxn = cobra.Reaction('BIODIL-ProtBiomM2C')
#model.add_reaction(rxn)
#rxn.reaction = 'BIO-modeledProtBiom_m --> BIO-modeledProtBiom_c'

model.reactions.get_by_id('DMRXNS-BIO-modeledProtBiom_m').bounds = (0,0)
model.reactions.get_by_id('DMRXNS-BIO-modeledProtBiom_c').bounds = (0,0)
model.reactions.get_by_id('DMRXNS-BIO-unmodeledProtBiom').bounds = (0,0)

model.reactions.get_by_id('BIODIL-BIODILFERMENTATION').bounds = (0,0)
#model.reactions.get_by_id('BIODIL-BIODILRESPIRATION').bounds = (0,0)

Using license file /home/hvdinh16/Workspace/Softwares/gurobi910/linux64/gurobi.lic
Academic license - for non-commercial use only - expires 2021-07-26


In [5]:
with open('./models/symbolicStoich_lower_0_211.json') as f:
    coeffs_low = json.load(f)
with open('./models/symbolicStoich_higher_0_211.json') as f:
    coeffs_high = json.load(f)
    
coeffs_low = {tuple(k.split(' | ')):v for k,v in coeffs_low.items()}
coeffs_high = {tuple(k.split(' | ')):v for k,v in coeffs_high.items()}

#### Binary search

In [6]:
glc = 1000
model.reactions.get_by_id('RXN-EX_glc__D_e_REV-SPONT').bounds = (0,glc)
model.reactions.get_by_id('RXN-EX_glc__D_e_FWD-SPONT').bounds = (0,0)
model.reactions.get_by_id('DMRXNS-BIO-rna_c').bounds = (0,0)

In [7]:
mu_min0 = 0; mu_max0 = 0.42; mu_tol = 1e-5; maxiter = 50;
mu_min = mu_min0; mu_max = mu_max0; itercount = 0;

class bcolors:
    GREEN = '\033[92m' #GREEN
    RED = '\033[91m' #RED
    RESET = '\033[0m' #RESET COLOR

# Test evaluation at zero
coeffs = selectCoeffSet(0, 0.211, coeffs_low, coeffs_high)
fba = evalModelAtMu(model, coeffs, 0)
if fba.status == 'infeasible':
    print('Model is infeasible at mu = 0, check model connectivity')

# Start binary search
# Evaluate min feasibility
mu = mu_min;
coeffs = selectCoeffSet(mu, 0.211, coeffs_low, coeffs_high)
fba = evalModelAtMu(model, coeffs, mu)
if fba.status == 'infeasible':
    mu_max = mu_min
    mu_min = 0
    print(f"{bcolors.RED}mu = {mu:.7f}, status = {fba.status}{bcolors.RESET}")
    
# Evaluate max infeasibility
mu = mu_max;
coeffs = selectCoeffSet(mu, 0.211, coeffs_low, coeffs_high)
fba = evalModelAtMu(model, coeffs, mu)
while fba.status == 'optimal':
    print(f"{bcolors.GREEN}mu = {mu:.7f}, status = {fba.status}{bcolors.RESET}")
    mu_max = 1.5*mu_max
    mu = mu_max;
    coeffs = selectCoeffSet(mu, 0.211, coeffs_low, coeffs_high)
    fba = evalModelAtMu(model, coeffs, mu)

# Update min-max
mu = float(mu_min + mu_max) / 2; fba_final = None;
while mu_max - mu_min > mu_tol and itercount < maxiter:
    itercount += 1
        
    coeffs = selectCoeffSet(mu, 0.211, coeffs_low, coeffs_high)
    fba = evalModelAtMu(model, coeffs, mu)
    if fba.status == 'optimal':
        mu_min = mu
        fba_final = fba
        print(f"{bcolors.GREEN}mu = {mu:.7f}, status = {fba.status}{bcolors.RESET}")
    else:
        mu_max = mu
        print(f"{bcolors.RED}mu = {mu:.7f}, status = {fba.status}{bcolors.RESET}")
        
    mu = float(mu_min + mu_max) / 2

/home/hvdinh16/Workspace/workpy3/lib/python3.7/site-packages/cobra/util/solver.py:430: UserWarning: solver status is 'infeasible'
  warn("solver status is '{}'".format(status), UserWarning)


mu = 0.2100000, status = optimal
mu = 0.3150000, status = infeasible
mu = 0.2625000, status = infeasible
mu = 0.2362500, status = infeasible
mu = 0.2231250, status = infeasible
mu = 0.2165625, status = infeasible
mu = 0.2132812, status = infeasible
mu = 0.2116406, status = infeasible
mu = 0.2108203, status = infeasible
mu = 0.2104102, status = optimal
mu = 0.2106152, status = optimal
mu = 0.2107178, status = infeasible
mu = 0.2106665, status = infeasible
mu = 0.2106409, status = infeasible
mu = 0.2106281, status = optimal
mu = 0.2106345, status = optimal


In [9]:
make_escher_csv(fba_final, './test.escher.csv')

In [6]:
glc = 1000
model.reactions.get_by_id('RXN-EX_glc__D_e_REV-SPONT').bounds = (0,glc)
model.reactions.get_by_id('RXN-EX_glc__D_e_FWD-SPONT').bounds = (0,0)
#model.reactions.get_by_id('DMRXNS-BIO-rna_c').bounds = (0,0)

In [7]:
mu_min0 = 0; mu_max0 = 0.42; mu_tol = 1e-5; maxiter = 50;
mu_min = mu_min0; mu_max = mu_max0; itercount = 0;

class bcolors:
    GREEN = '\033[92m' #GREEN
    RED = '\033[91m' #RED
    RESET = '\033[0m' #RESET COLOR

# Test evaluation at zero
coeffs = selectCoeffSet(0, 0.211, coeffs_low, coeffs_high)
fba = evalModelAtMu(model, coeffs, 0)
if fba.status == 'infeasible':
    print('Model is infeasible at mu = 0, check model connectivity')

# Start binary search
# Evaluate min feasibility
mu = mu_min;
coeffs = selectCoeffSet(mu, 0.211, coeffs_low, coeffs_high)
fba = evalModelAtMu(model, coeffs, mu)
if fba.status == 'infeasible':
    mu_max = mu_min
    mu_min = 0
    print(f"{bcolors.RED}mu = {mu:.7f}, status = {fba.status}{bcolors.RESET}")
    
# Evaluate max infeasibility
mu = mu_max;
coeffs = selectCoeffSet(mu, 0.211, coeffs_low, coeffs_high)
fba = evalModelAtMu(model, coeffs, mu)
while fba.status == 'optimal':
    print(f"{bcolors.GREEN}mu = {mu:.7f}, status = {fba.status}{bcolors.RESET}")
    mu_max = 1.5*mu_max
    mu = mu_max;
    coeffs = selectCoeffSet(mu, 0.211, coeffs_low, coeffs_high)
    fba = evalModelAtMu(model, coeffs, mu)

# Update min-max
mu = float(mu_min + mu_max) / 2; fba_final = None;
while mu_max - mu_min > mu_tol and itercount < maxiter:
    itercount += 1
        
    coeffs = selectCoeffSet(mu, 0.211, coeffs_low, coeffs_high)
    fba = evalModelAtMu(model, coeffs, mu)
    if fba.status == 'optimal':
        mu_min = mu
        fba_final = fba
        print(f"{bcolors.GREEN}mu = {mu:.7f}, status = {fba.status}{bcolors.RESET}")
    else:
        mu_max = mu
        print(f"{bcolors.RED}mu = {mu:.7f}, status = {fba.status}{bcolors.RESET}")
        
    mu = float(mu_min + mu_max) / 2

mu = 0.4200000, status = optimal
mu = 0.6300000, status = optimal


/home/hvdinh16/Workspace/workpy3/lib/python3.7/site-packages/cobra/util/solver.py:430: UserWarning: solver status is 'infeasible'
  warn("solver status is '{}'".format(status), UserWarning)


mu = 0.4725000, status = optimal
mu = 0.7087500, status = infeasible
mu = 0.5906250, status = optimal
mu = 0.6496875, status = infeasible
mu = 0.6201562, status = optimal
mu = 0.6349219, status = optimal
mu = 0.6423047, status = optimal
mu = 0.6459961, status = infeasible
mu = 0.6441504, status = optimal
mu = 0.6450732, status = optimal
mu = 0.6455347, status = infeasible
mu = 0.6453040, status = optimal
mu = 0.6454193, status = infeasible
mu = 0.6453616, status = optimal
mu = 0.6453905, status = optimal
mu = 0.6454049, status = optimal
mu = 0.6454121, status = infeasible


In [8]:
make_escher_csv(fba_final, './test_noLimit_rRNA.escher.csv')

In [10]:
print('Growth rate:', fba_final.fluxes['BIODIL-BIODILRESPIRATION'])
print('Ethanol:', fba_final.fluxes['RXN-EX_etoh_e_FWD-SPONT'])
for rxn in fba_final.fluxes.index:
    if rxn[:7] == 'RXN-EX_' and fba_final.fluxes[rxn] > 0:
        print(rxn, fba_final[rxn])

Growth rate: 0.5807128906249999
Ethanol: 0.0
RXN-EX_nh4_e_REV-SPONT 4.5886981276815435
RXN-EX_co2_e_FWD-SPONT 17.052075763881046
RXN-EX_glc__D_e_REV-SPONT 6.913949357429679
RXN-EX_gcald_e_FWD-SPONT 6.13128510704866e-05
RXN-EX_h_e_FWD-SPONT 1.6022524937540257
RXN-EX_fe2_e_REV-SPONT 0.0003263360264448513
RXN-EX_o2_e_REV-SPONT 16.20132383854176
RXN-EX_pi_e_REV-SPONT 0.3908456937476686
RXN-EX_k_e_REV-SPONT 0.13969527873621954
RXN-EX_so4_e_REV-SPONT 0.15851320767951474
RXN-EX_h2o_e_FWD-SPONT 29.7237158101146
RXN-EX_cu2_e_REV-SPONT 8.58022324830105e-05
RXN-EX_mn2_e_REV-SPONT 2.6051925646674407e-05
RXN-EX_zn2_e_REV-SPONT 0.0005906148532357512
RXN-EX_mg2_e_REV-SPONT 0.014983419046922093
RXN-EX_ca2_e_REV-SPONT 0.0002973249999999999
RXN-EX_cobalt2_e_REV-SPONT 1.3348413106883243e-05
RXN-EX_ni2_e_REV-SPONT 2.617396632826903e-06


In [15]:
df_rxns = pd.read_excel('../scRBA_new/scRAM_network_frompy.xlsx')

In [16]:
df_rxns[df_rxns.id.str.contains('ETOHt_c_e')]

,id,type,symbolic_coeffs,reaction
1709,RXN-2PHETOHt_c_e_FWD-SPONT,metabolic,NaN,MET-2phetoh_e --> MET-2phetoh_c
1710,RXN-2PHETOHt_c_e_REV-SPONT,metabolic,NaN,MET-2phetoh_c --> MET-2phetoh_e
1928,RXN-ETOHt_c_e_FWD-SPONT,metabolic,NaN,MET-etoh_c --> MET-etoh_e
1929,RXN-ETOHt_c_e_REV-SPONT,metabolic,NaN,MET-etoh_e --> MET-etoh_c


Test model evaluation

In [6]:
mu = 0.1
if mu < 0.211:
    coeffs = deepcopy(coeffs_low)
else:
    coeffs = deepcopy(coeffs_high)
fba = evalModelAtMu(model, coeffs, mu)

In [7]:
fba

,fluxes,reduced_costs
RXN-BTDDH_c_FWD-YAL060W,0.000000,-6.696900e-15
RXN-BTDDH_c_REV-YAL060W,0.000000,-3.979008e+00
RXN-13GS_c_FWD-GSC2RHO1,0.000000,-4.190353e-04
RXN-13GS_c_FWD-GSC1RHO1,0.098991,-1.708789e-14
RXN-16GS_c_FWD-YPR159W,0.026050,-1.730257e-14
...,...,...
BIODIL-BIODILRESPIRATION,0.100000,-2.174486e+03
DMRXNS-BIO-unmodeledProtBiom,0.000000,-1.758100e+03
DMRXNS-BIO-modeledProtBiom_c,0.000000,-1.758100e+03
DMRXNS-BIO-modeledProtBiom_m,0.000000,-1.758100e+03
